In [35]:


url = "https://www.booking.com/searchresults.de.html?label=gen173nr-1BCAEoggI46AdIM1gEaDuIAQGYAQe4ARfIAQzYAQHoAQGIAgGoAgO4AuSg57oGwAIB0gIkN2YxNzc5MjMtNDhlNy00MzBiLWFhODgtMjllZWEyMGQ0ZThk2AIF4AIB&sid=4daa01aeadda31e7f02b8ddc59e9f35f&aid=304142&ss=Kreta&ssne=Kreta&ssne_untouched=Kreta&efdco=1&lang=de&src=searchresults&dest_id=811&dest_type=region&checkin=2025-03-24&checkout=2025-03-29&ltfd=%3A%3A8-2025%3A1%3A&group_adults=2&no_rooms=1&group_children=0&nflt=ht_id%3D204"

In [37]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import random

def handle_no_such_element_exception(data_extraction_task):
    try:
        return data_extraction_task()
    except NoSuchElementException as e:
        return None

# Create a Chrome web driver instance
driver = webdriver.Chrome(service=Service())

# Connect to the target page
driver.get(url)

# Handle the sign-in alert
try:
    close_button = WebDriverWait(driver, 12).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "[role=\"dialog\"] button[aria-label=\"Dismiss sign-in info.\"]"))
    )
    close_button.click()
except Exception as e:
    print("Sign-in modal did not appear, continuing...")

# Scroll down the page in a human-like fashion with a maximum scroll time
scroll_pause_time = 3  # Pause time between scrolls
max_scroll_time = 320  # Maximum scroll time in seconds
start_time = time.time()
last_height = driver.execute_script("return document.body.scrollHeight")
load_more_button_found = False
max_button_clicks = 30  # Maximum number of button clicks

while True:
    # Scroll down to the bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load the page
    time.sleep(scroll_pause_time)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height or (time.time() - start_time) > max_scroll_time:
        break
    last_height = new_height

    # Check if "Load More Results" button is present and click it
    if max_button_clicks > 0:
        try:
            load_more_button = WebDriverWait(driver, 6).until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, "button.a83ed08757.c21c56c305.bf0537ecb5.f671049264.af7297d90d.c0e0affd09")
                )
            )
            load_more_button.click()
            load_more_button_found = True
            print("Clicked 'Load More Results' button")
            # Add a small pause after clicking the button
            time.sleep(2)
            max_button_clicks -= 1
        except TimeoutException:
            pass  # Button not yet available, continue scrolling

# If "Load More Results" button was found, wait an additional time for final loading
if load_more_button_found:
    time.sleep(5)  # Adjust this wait time as needed

# Where to store the scraped data
items = []


# Select all property items on the page
property_items = driver.find_elements(By.CSS_SELECTOR, "[data-testid=\"property-card\"]")

print(f"Found {len(property_items)} property items.")


Sign-in modal did not appear, continuing...
Found 71 property items.


In [38]:

import csv

for property_item in property_items:
    # scraping logic...
    url = handle_no_such_element_exception(lambda: property_item.find_element(By.CSS_SELECTOR, "a[data-testid=\"property-card-desktop-single-image\"]").get_attribute("href"))
    image = handle_no_such_element_exception(lambda: property_item.find_element(By.CSS_SELECTOR, "img[data-testid=\"image\"]").get_attribute("src"))
    time.sleep(random.uniform(1, 2))
    title = handle_no_such_element_exception(lambda: property_item.find_element(By.CSS_SELECTOR, "[data-testid=\"title\"]").text)
    address = handle_no_such_element_exception(lambda: property_item.find_element(By.CSS_SELECTOR, "[data-testid=\"address\"]").text)
    distance = handle_no_such_element_exception(lambda: property_item.find_element(By.CSS_SELECTOR, "[data-testid=\"distance\"]").text)

    review_score = None
    review_count = None
    review_text = handle_no_such_element_exception(lambda: property_item.find_element(By.CSS_SELECTOR, "[data-testid=\"review-score\"]").text)
    if review_text is not None:
    # split the review string by \n
        parts = review_text.split("\n")

    # process each part
    for part in parts:
        part = part.strip()
        # check if this part is a number (potential review score)
        if part.replace(".", "", 1).isdigit():
            review_score = float(part)
        # check if it contains the "reviews" string
        elif "reviews" in part:
            # extract the number before "reviews"
            review_count = int(part.split(" ")[0].replace(",", ""))
    decription = handle_no_such_element_exception(lambda: property_item.find_element(By.CSS_SELECTOR, "[data-testid=\"recommended-units\"]").text)
    time.sleep(random.uniform(1, 2))
    price_element = handle_no_such_element_exception(lambda: (property_item.find_element(By.CSS_SELECTOR, "[data-testid=\"availability-rate-information\"]")))
    if price_element is not None:
        original_price = handle_no_such_element_exception(lambda: (
            price_element.find_element(By.CSS_SELECTOR, "[aria-hidden=\"true\"]:not([data-testid])").text.replace(",", "")
        ))
        price = handle_no_such_element_exception(lambda: (
            price_element.find_element(By.CSS_SELECTOR, "[data-testid=\"price-and-discounted-price\"]").text.replace(",", "")
        ))

    # populate a new item with the scraped data
    item = {
    "url": url,
    "image": image,
    "title": title,
    "address": address,
    "distance": distance,
    "review_score": review_score,
    "review_count": review_count,
    "decription": decription,
    "original_price": original_price,
    "price": price
    }
    # add the new item to the list of scraped items
    items.append(item)

output_file = "properties2.csv"

with open(output_file, mode="w", newline="", encoding="utf-8") as file:
        #create a CSV writer object
    writer = csv.DictWriter(file, fieldnames=["url", "image", "title", "address", "distance", "review_score", "review_count", "decription", "original_price", "price"])

    # write the header row
    writer.writeheader()

    # write each item as a row in the CSV
    writer.writerows(items)
    driver.quit()


In [39]:
import re


hotel_url_names = []

for item in items:
    url = item['url']
    match = re.search(r'hotel/gr/(.+?)\.de\.html', url)
    if match:
        hotel_url_name = match.group(1)
        hotel_url_names.append(hotel_url_name)

print(hotel_url_names)

['heliades-residence', 'vaya-suites-by-omilos-hotels', 'lyktos', 'alsus-boutique-hotel-adults-only', 'manili', 'georgia', 'centro-storico', 'elounda-colour-apartments-elounta', 'sapphire-horizon-suites', 'infinity-wellness-boutique', 'panorama-villas-aghios-nikolaos', 'veneziano-boutique', 'infinity-city-boutique', 'royal-sun', 'glaros-beach', 'alexis', 'inotropes-boutique-suites', 'notus', 'mansio-boutique', 'above-suites', 'thalassa-boutique-hotel-rethimno', 'galaxy-iraklio', 'cretan-berry-kondilaki', 'dom-boutique', 'santo-george-beach', 'villa-mare-monte-studios-and-apartments', 'arkadi', 'christina-chania', 'hyperion-city', 'montre-fine-living-agios-nikolaos', 'sea-breeze-hotel', 'planis-str-urban-suites', 'irida-khania', 'civitas-rethymnae', 'petousis-amp-suites', 'royal-gold-city-suites', 'atrionhotelheraklion', 'pietra-di-mare', 'koum-kapi', 'kyma-suites-beach-hotel', 'elysee', 'polyxenia-petres', 'nek-suites', 'vespera-city-erakleio', 'iraklion', 'veranda-rossa', 'epavli-chani

In [40]:
# ADD keys
import pandas as pd
df = pd.DataFrame(items)




In [68]:
######
#Geofunctions
import pydeck as pdk

from geopy.geocoders import Nominatim

# Function to get geolocation data
def get_geolocation(description):
    geolocator = Nominatim(user_agent="geo_app")
    location = geolocator.geocode(f"{description}, Crete" , timeout=10)
    if location:
        return location.latitude, location.longitude
    return None, None

# Function to try multiple geocoding strategies
def find_location(description):
    # Try the full description first
    latitude, longitude = get_geolocation(description)
    if latitude is not None and longitude is not None:
        return latitude, longitude
    
    # If not found, simplify the description and try again
    #print("simplefied")
    simplified_description = description.split(',')[-1].strip()
    return get_geolocation(simplified_description)



In [ ]:
df['latitude'], df['longitude'] = zip(*df['address'].apply(find_location))


Note: you may need to restart the kernel to use updated packages.


In [79]:
zip(*df['address'][6:7].apply(find_location))
#df['address'][5:7]
#geolocator = Nominatim(user_agent="geo_app")
#description = "Chania Altstadt, Chania"

#find_location(description)

None
simplefied
Χανιά, Δήμος Χανίων, Περιφερειακή Ενότητα Χανίων, Περιφέρεια Κρήτης, Αποκεντρωμένη Διοίκηση Κρήτης, 731 36, Ελλάς


In [33]:
df["Hotel_key"] = (hotel_url_names)
print(df)
df.to_json('properties_gr2.json', orient='records', lines=True)

NameError: name 'hotel_url_names' is not defined

In [5]:
for hotel_url_name in hotel_url_names:
    !python run.py {hotel_url_name} "gr"


2024-12-12 10:01:43,777 - root - INFO - 

******** Input Params ********
-->  country: gr
-->  hotel_name: the-chania
-->  sort_by: most_relevant
-->  n_rows: -1
-->  stop_critera: None
************************


2024-12-12 10:01:43,777 - root - INFO - Creating URLs
2024-12-12 10:01:43,777 - root - INFO - Checking max offset parameter value
2024-12-12 10:01:44,093 - root - INFO - Offset parameter max value: 670
2024-12-12 10:01:44,099 - root - INFO - Created URLs: 68
2024-12-12 10:01:44,099 - root - INFO - Progress Monitoring Thread Started
2024-12-12 10:01:44,099 - root - INFO - Starting Get Requests on 68 urls
2024-12-12 10:01:50,488 - root - INFO - Finished Get Requests in 6.4 seconds
2024-12-12 10:01:50,488 - root - INFO - Starting Parsing Responses: 68
2024-12-12 10:01:52,104 - root - INFO - Processed 15/68
2024-12-12 10:01:54,105 - root - INFO - Processed 55/68
2024-12-12 10:01:54,301 - root - INFO - Processes launched: 5
2024-12-12 10:01:54,821 - root - INFO - Finished Parsing R

Process Process-4:
Traceback (most recent call last):
  File "c:\Users\alan_\anaconda3\envs\Scraping\Lib\multiprocessing\process.py", line 314, in _bootstrap
    self.run()
  File "c:\Users\alan_\anaconda3\envs\Scraping\Lib\multiprocessing\process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "c:\GitHub Projects\Hotel_review_project\Scraping\core\scrape.py", line 432, in _parse_scraped_results
    if "en" in original_lang:
       ^^^^^^^^^^^^^^^^^^^^^
TypeError: argument of type 'NoneType' is not iterable


2024-12-12 10:42:38,231 - root - INFO - 

******** Input Params ********
-->  country: gr
-->  hotel_name: cathedral-suites-adults-only
-->  sort_by: most_relevant
-->  n_rows: -1
-->  stop_critera: None
************************


2024-12-12 10:42:38,231 - root - INFO - Creating URLs
2024-12-12 10:42:38,231 - root - INFO - Checking max offset parameter value
2024-12-12 10:42:38,637 - root - INFO - Offset parameter max value: 80
2024-12-12 10:42:38,638 - root - INFO - Created URLs: 9
2024-12-12 10:42:38,639 - root - INFO - Progress Monitoring Thread Started
2024-12-12 10:42:38,639 - root - INFO - Starting Get Requests on 9 urls
2024-12-12 10:42:39,008 - root - INFO - Finished Get Requests in 0.4 seconds
2024-12-12 10:42:39,008 - root - INFO - Starting Parsing Responses: 9
2024-12-12 10:42:40,640 - root - INFO - Processed 4/9
2024-12-12 10:42:42,629 - root - INFO - Processes launched: 5
2024-12-12 10:42:42,641 - root - INFO - Processed 8/9
2024-12-12 10:42:42,702 - root - INFO - Finished